# Modelo SEIR

O modelo SEIR consiste em resolver as seguintes equações diferenciais:
$$\begin{aligned}
\frac{d \mathcal{S}}{d t} &=-\frac{r_{0}}{T_{\mathrm{inf}}} \mathcal{S} \mathcal{I} \\
\frac{d \mathcal{E}}{d t} &=\frac{r_{0}}{T_{\mathrm{inf}}} \mathcal{S} \mathcal{I}-\frac{1}{T_{\mathrm{inc}}} \mathcal{E} \\
\frac{d \mathcal{I}}{d t} &=\frac{1}{T_{\mathrm{inc}}} \mathcal{E}-\frac{1}{T_{\mathrm{inf}}} \mathcal{I} \\
\frac{d \mathcal{R}}{d t} &=\frac{1}{T_{\mathrm{inf}}} \mathcal{I}
\end{aligned}$$

Sabendo que, segundo os dados do enunciado, tem-se os seguintes valores:
- Número de reprodução basal: $r_0=2,5$
- Tempos médio de incubação e infecção: $T_{\mathrm{inc}}=5,2\text{ dias}$ e $T_{\mathrm{inf}}=2,9\text{ dias}$


In [8]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
O código bruto do Notebook é, por padrão, escondido para facilitar a leitura das informações.
Para fazer aparecer/desaparecer o código, só clicar <a href="javascript:code_toggle()">aqui</a>.''')

In [24]:
import ipywidgets as widgets
from IPython.display import Javascript, display

html = widgets.HTMLMath(
    value=r"Para rodar as simulações, basta mudar os valores das condições iniciais com os sliders abaixo. Por fim, é importante clicar no botão para que as simulações sejam compiladas",
    placeholder='Some HTML',
    description='',
)

slider = widgets.FloatSlider(
    min=0,
    max=10,
    step=0.5,
    description='\(r_0\)',
    value=2.5,
    readout=True,
    readout_format='.1f',
)
slider1 = widgets.FloatSlider(
    min=0,
    max=10,
    step=0.5,
    description='\(T_{inc}\)',
    value=5.2,
    readout=True,
    readout_format='.1f',
)
slider2 = widgets.FloatSlider(
    min=0,
    max=10,
    step=0.5,
    description='\(T_{inf}\)',
    value=2.9,
    readout=True,
    readout_format='.1f',
)
checkbox = widgets.Checkbox(
    value=False,
    description='Deseja que \(r_0\) seja aleatório?',
    disabled=False
)
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

button = widgets.Button(description="Rodar as Simulações")
button.on_click(run_all)

display(html)
display(slider,checkbox,slider1, slider2)
display(button)

HTMLMath(value='Para rodar as simulações, basta mudar os valores das condições iniciais com os sliders abaixo.…

FloatSlider(value=2.5, description='\\(r_0\\)', max=10.0, readout_format='.1f', step=0.5)

Checkbox(value=False, description='Deseja que \\(r_0\\) seja aleatório?')

FloatSlider(value=5.2, description='\\(T_{inc}\\)', max=10.0, readout_format='.1f', step=0.5)

FloatSlider(value=2.9, description='\\(T_{inf}\\)', max=10.0, readout_format='.1f', step=0.5)

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
import numpy as np
%matplotlib widget
#%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set();

r_0=slider.value #número de reprodução basal
T_inc=slider1.value #tempo de incubação
T_inf=slider2.value #tempo de infecção
tf=498 #número final de dias da simulação
dt=1 # passo (1 dia)
t=np.arange(0,tf+0.5,dt) # vetor com todos os dias até o 498º dia
n=len(t) #tamanho do vetor t => número de dias da simulação

S = np.ones(n)*0.999999
E = np.ones(n)*(10**-6)
I = np.zeros(n)
R = np.zeros(n)

def f(y):
    a = (r_0/T_inf)*y[0]*y[2]
    b = (1/T_inc)*y[1]
    c = (1/T_inf)*y[2]
    S_ = -a
    E_ = (a - b)
    I_ = (b - c)
    R_ = c
    return [S_,E_,I_,R_]

rand=[]
for i in range(1,n):
    if checkbox.value:
        a=np.random.rand(1)[0]*5
        r_0=abs(a*np.cos(2.5*np.pi*t[i]/498))+0.5;rand.append(r_0)
    #r_0=np.random.rand(1)[0]*4;rand.append(r_0)
    #if i>=50:
    #    r_0=(2.5+1)/i
    y05 = f([S[i-1],E[i-1],I[i-1],R[i-2]])
    S_05 = S[i-1] + y05[0]*(dt/2)
    E_05 = E[i-1] + y05[1]*(dt/2)
    I_05 = I[i-1] + y05[2]*(dt/2)
    R_05 = R[i-1] + y05[3]*(dt/2)
    y_ = [S_05,E_05,I_05,R_05]
    S[i] = S[i-1] + dt*f(y_)[0]
    E[i] = E[i-1] + dt*f(y_)[1]
    I[i] = I[i-1] + dt*f(y_)[2]
    R[i] = R[i-1] + dt*f(y_)[3]

#print(S)
#print(E)
#print(I)
#print(R)
#print(rand)

In [47]:
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display
from IPython.display import HTML


# Layout
layout = go.Layout(
    title="Proporção do modelo SEIR versus tempo",
    title_x=0.5,
    xaxis=dict(
        title="Tempo (dias)"
        ),
        yaxis=dict(
        title="Porcentagem da População (%)"
        ) 
)
fig = go.Figure(layout = layout)

# Add scatter trace with medium sized markers
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=S,
        name='Suscetíveis',
        marker=dict(
            color='darkturquoise',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=E,
        name='Expostos',
        marker=dict(
            color='darkmagenta',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=I,
        name='Infectados',
        marker=dict(
            color='red',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=R,
        name='Recuperados',
        marker=dict(
            color='yellow',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)

In [48]:
S0 = np.ones(n)
E0 = np.ones(n)
I0 = np.zeros(n)
R0 = np.zeros(n)
L0 = np.zeros(n)
P0 = np.zeros(n)
W0 = np.zeros(n)

for j in range(n):
    S0[j] = int(S[j]*44040000)
    E0[j] = int(E[j]*44040000)
    I0[j] = int(I[j]*44040000)
    R0[j] = int(R[j]*44040000)
    L0[j] = int((27*44040000)/100000)
    if j>=1:
        P0[j] = int(abs(R0[j-1]-R0[j])*0.03)
    
for p in range(n):
    if p==0:
        W0[p]=P0[p]
    elif p==1:
        W0[p]=P0[p]+P0[p-1]
    elif p==2:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]
    elif p==3:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]+P0[p-3]
    elif p==4:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]+P0[p-3]+P0[p-4]
    elif p==5:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]+P0[p-3]+P0[p-4]+P0[p-5]
    else:
        W0[p]=P0[p]+P0[p-1]+P0[p-2]+P0[p-3]+P0[p-4]+P0[p-5]+P0[p-6]
    
        
# Layout
layout = go.Layout(
    title="Proporção do modelo SEIR versus tempo",
    title_x=0.5,
    xaxis=dict(
        title="Tempo (dias)"
        ),
        yaxis=dict(
        title="Porcentagem da População (%)"
        ) 
)
fig = go.Figure(layout = layout)

# Add scatter trace with medium sized markers
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=S0,
        name='Suscetíveis',
        marker=dict(
            color='darkturquoise',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=E0,
        name='Expostos',
        marker=dict(
            color='darkmagenta',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=I0,
        name='Infectados',
        marker=dict(
            color='red',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=R0,
        name='Recuperados',
        marker=dict(
            color='yellow',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=L0,
        name='Leitos',
        marker=dict(
            color='orange',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=W0,
        name='Necessidade de Leitos',
        marker=dict(
            color='green',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=rand,
        name='Reprodução basal',
        marker=dict(
            color='black',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)